Below is the sample interface of this program.

![title](sample.png)

In [1]:
import pandas as pd

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State

from plotly.subplots import make_subplots
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv('USROP_A 0 N-NA_F-9_Ad.csv')
df.drop(df.columns[0], axis=1, inplace =True)
df.set_index(df.columns[0], inplace =True)

list_of_parameters = df.columns

parameter_options = []
for param in list_of_parameters:
    parameter_options.append({'label':str(param),'value':param})
    
df

,Weight on Bit kkgf,Average Standpipe Pressure kPa,Average Surface Torque kN.m,Rate of Penetration m/h,Average Rotary Speed rpm,Mud Flow In L/min,Mud Density In g/cm3,Diameter mm,Average Hookload kkgf,Hole Depth (TVD) m,USROP Gamma gAPI
Measured Depth m,,,,,,,,,,,
491.033,5.842270,9440.922214,0.244047,42.864024,84.000,2784.321942,1.210000,311.15,93.780222,490.760309,150.88
491.185,6.241431,9499.941336,0.244047,42.315384,84.000,2784.321942,1.210000,311.15,93.979803,490.910880,150.88
491.222,6.241431,9499.941336,0.244047,42.315384,84.000,2784.321942,1.210000,311.15,93.979803,490.910880,150.88
491.338,6.368437,9313.024466,0.216931,42.894504,84.000,2784.321942,1.210000,311.15,94.079593,491.061756,146.26
491.341,6.368437,9313.024466,0.216931,42.894504,84.000,2784.321942,1.210000,311.15,94.079593,491.061756,146.26
...,...,...,...,...,...,...,...,...,...,...,...
1205.755,12.455646,10986.726799,9.666982,40.614600,194.680,1927.456082,1.198264,215.90,88.119390,1013.078086,25.37
1205.789,12.407112,10953.887070,9.597835,39.549934,194.571,1927.467438,1.198264,215.90,88.213283,1013.095459,25.37
1205.810,12.407112,10953.887070,9.597835,39.549934,194.571,1927.467438,1.198264,215.90,88.213283,1013.106432,25.37


In [ ]:
app = dash.Dash()

app.layout = html.Div([
    html.Div([
            dcc.Dropdown(id='param_option',
            options=parameter_options,
            value=['Mud Density In g/cm3','Average Standpipe Pressure kPa','Average Surface Torque kN.m','Rate of Penetration m/h'],
            multi=True),
        
        
            html.Button(id='my_button', 
            n_clicks=0, 
            children='Submit', 
            style={'fontSize':28,'display':'inline-block'})
        
            ]),
    
    html.Div([
            dcc.Graph(id='my_graph')
            ])#,style={'width':'10%',height='display':'inline-block'})
        
            ])
    
    
    
  
@app.callback(
        Output('my_graph', 'figure'),
        [Input('my_button', 'n_clicks')], 
        [State('param_option', 'value')])
    
def update_figure(nclicks, params):
        if len(params)<6:
            fig = make_subplots(rows=1, cols=len(params),shared_yaxes=True)
            for param in params:
                 fig.add_trace(go.Scatter(x = df[param] , y = df.index, mode = 'lines', name = param),row=1, col=params.index(param)+1)
                 fig.update_xaxes(title_text=param, row=1, col=params.index(param)+1) 
            fig.update_yaxes(title_text=df.index.name, row=1, col=1) 
            
            
        if len(params)>5:
            fig = make_subplots(rows=2, cols=5,shared_yaxes=True)
            
            for param in params:
                if params.index(param) < 5:
                     fig.add_trace(go.Scatter(x = df[param] , y = df.index, mode = 'lines', name = param),row=1, col=params.index(param)+1)
                     fig.update_xaxes(title_text=param, row=1, col=params.index(param)+1)
                else:
                     fig.add_trace(go.Scatter(x = df[param] , y = df.index, mode = 'lines', name = param),row=2, col=params.index(param)-4)
                     fig.update_xaxes(title_text=param, row=2, col=params.index(param)-4)
            fig.update_yaxes(title_text=df.index.name, row=1, col=1)         
            fig.update_yaxes(title_text=df.index.name, row=2, col=1) 
            
        #if len(params)>3:
         #   fig.add_trace(go.Scatter(x = df[params[3]] , y = df.index, mode = 'lines', name = params[3]),row=1, col=3)
        fig.update_yaxes(title_text=df.index.name, row=1, col=1)   
        fig.update_yaxes(autorange="reversed")
        return fig



if __name__ == '__main__': 
    app.run_server()